In [ ]:
import os
import pandas as pd
import db_mysql as db
import re

def carica_singoli_da_cartelle(base_dir):
    """
    Carica i file xlsx presenti nelle sottocartelle di una cartella base in un DataFrame.
    
    Argomenti:
    base_dir (str): Percorso della cartella base.
    
    Ritorno:
    dict: Un dizionario in cui le chiavi sono i nomi delle cartelle e i valori sono i DataFrame corrispondenti ai file xlsx.
    """
    dati = {}
    for cartella in os.listdir(base_dir):
        cartella_path = os.path.join(base_dir, cartella)
        if os.path.isdir(cartella_path):
            dati_cartella = {}
            for file in os.listdir(cartella_path):
                file_path = os.path.join(cartella_path, file)
                if file.endswith('.xlsx'):
                    df = pd.read_excel(file_path, sheet_name='Singoli', header=6)
                    dati_cartella[file] = df
            dati[cartella] = dati_cartella
    return dati

# utilizzo:
cartella_base = 'Iscrizioni'
dati = carica_singoli_da_cartelle(cartella_base)

In [ ]:
df_singoli = pd.DataFrame()
for cartella, dati_cartella in dati.items():
    for file, df in dati_cartella.items():
        df.insert(1, 'SOCIETA', cartella)
        df_singoli = pd.concat([df_singoli, df], axis=0)

df_singoli = df_singoli.rename(columns={'ANNO NASCITA [2009-2018]': 'ANNO NASCITA'})
df_singoli = df_singoli.drop(columns=['Unnamed: 0', 'QUOTA DI ISCRIZIONE', 'CONVALIDA FINALE'])
df_singoli.index = df_singoli.index + 1
df_singoli.index.name = 'N_ISCRITTO_SOCIETA'
df_singoli = df_singoli.reset_index()
df_singoli.index = df_singoli.index + 1
df_singoli.index.name = 'ID_ISCRITTO'

In [ ]:
df_singoli.to_parquet('Importati/singoli.parquet')
df_singoli

In [ ]:
import os
import pandas as pd

def carica_squadre_da_cartelle(base_dir):
    """
    Carica i file xlsx presenti nelle sottocartelle di una cartella base in un DataFrame.
    
    Argomenti:
    base_dir (str): Percorso della cartella base.
    
    Ritorno:
    dict: Un dizionario in cui le chiavi sono i nomi delle cartelle e i valori sono i DataFrame corrispondenti ai file xlsx.
    """
    dati = {}
    for cartella in os.listdir(base_dir):
        cartella_path = os.path.join(base_dir, cartella)
        if os.path.isdir(cartella_path):
            dati_cartella = {}
            for file in os.listdir(cartella_path):
                file_path = os.path.join(cartella_path, file)
                if file.endswith('.xlsx'):
                    df = pd.read_excel(file_path, sheet_name='Squadre', header=4)
                    dati_cartella[file] = df
            dati[cartella] = dati_cartella
    return dati

# utilizzo:
cartella_base = 'Iscrizioni'
dati = carica_squadre_da_cartelle(cartella_base)

In [ ]:
df_tot_squadra_A = pd.DataFrame()
df_tot_squadra_B = pd.DataFrame()
for cartella, dati_cartella in dati.items():
    for file, df in dati_cartella.items():
        df.insert(1, 'SOCIETA', cartella)
        squadra1 = df.iloc[0:3]
        squadra2 = df.iloc[9:12]
        df_tot_squadra_A = pd.concat([df_tot_squadra_A, squadra1], axis=0)
        df_tot_squadra_B = pd.concat([df_tot_squadra_B, squadra2], axis=0)

df_tot_squadra_A = df_tot_squadra_A.rename(columns={'ANNO NASCITA [2009-2018]': 'ANNO NASCITA'})
df_tot_squadra_A = df_tot_squadra_A.drop(columns=['Unnamed: 0', 'CONVALIDA'])
df_tot_squadra_A.index = df_tot_squadra_A.index + 1
df_tot_squadra_A.index.name = 'N_ISCRITTO_SOCIETA'
df_tot_squadra_A = df_tot_squadra_A.dropna(subset=['COGNOME'])
df_tot_squadra_A = df_tot_squadra_A.reset_index()
df_tot_squadra_A.index = df_tot_squadra_A.index + 1
df_tot_squadra_A.index.name = 'ID_ISCRITTO'

df_tot_squadra_B = df_tot_squadra_B.rename(columns={'ANNO NASCITA [2009-2018]': 'ANNO NASCITA'})
df_tot_squadra_B = df_tot_squadra_B.drop(columns=['Unnamed: 0', 'CONVALIDA'])
df_tot_squadra_B.index = df_tot_squadra_B.index + 1
df_tot_squadra_B.index.name = 'N_ISCRITTO_SOCIETA'
df_tot_squadra_B = df_tot_squadra_B.dropna(subset=['COGNOME'])
df_tot_squadra_B = df_tot_squadra_B.reset_index()
df_tot_squadra_B.index = df_tot_squadra_B.index + 1
df_tot_squadra_B.index.name = 'ID_ISCRITTO'

In [ ]:
df_tot_squadra_A.to_parquet('Importati/squadra_A.parquet')
df_tot_squadra_A

In [ ]:
df_tot_squadra_B.to_parquet('Importati/squadra_B.parquet')
df_tot_squadra_B

## Salva società su db

In [ ]:
società = df_singoli['SOCIETA'].str.upper().drop_duplicates().to_frame().rename(columns={'SOCIETA': 'id_societa'})
temp_df1 = df_tot_squadra_A['SOCIETA'].str.upper().drop_duplicates().to_frame().rename(columns={'SOCIETA': 'id_societa'})
temp_df2 = df_tot_squadra_B['SOCIETA'].str.upper().drop_duplicates().to_frame().rename(columns={'SOCIETA': 'id_societa'})
società = pd.concat([società, temp_df1, temp_df2], axis=0)
società['id_societa'] = società['id_societa'].str.upper()
società['nome'] = società['id_societa']
società['id_societa'] = società['id_societa'].apply(lambda x: re.sub(r'\s', '', x))
società.drop_duplicates(subset='id_societa', inplace=True)
società.set_index('id_societa', inplace=True)
società['n_ordine'] = range(1, len(società) + 1)

def salva_società(df):
    db.execute_query('DELETE FROM societa', None)
    for index, row in df.iterrows():
        # Aggiunta degli indici alla lista dei valori
        values_with_indices = [index] + [None if (pd.isna(value)) else value for value in row]
        
        # Aggiunta degli indici alla lista dei nomi delle colonne
        columns_with_indices = ['id_societa'] + df.columns.tolist()
        
        # Creazione della query di inserimento con i parametri
        sql = f"INSERT INTO societa ({', '.join(columns_with_indices)}) VALUES ({', '.join(['%s' for _ in range(len(values_with_indices))])})"
        
        # Esecuzione della query utilizzando la funzione execute_query
        db.execute_query(sql, tuple(values_with_indices))
        
salva_società(società)
società

## Salva atleti su db

In [47]:
atleti = df_singoli.drop(columns=['N_ISCRITTO_SOCIETA'])
atleti.rename(inplace=True, columns={'COGNOME': 'cognome', 'NOME': 'nome', 'SESSO': 'sesso', 'ANNO NASCITA': 'anno_nascita', 'CINTURA': 'cintura', 'SOCIETA': 'societa', 'CATEGORIA': 'categoria'})
atleti['sesso'] = atleti['sesso'].apply(lambda x: x[0].upper())
atleti['cognome'] = atleti['cognome'].apply(lambda x: x.upper())
atleti['nome'] = atleti['nome'].apply(lambda x: x.upper())
atleti['cintura'] = atleti['cintura'].apply(lambda x: x.upper())
atleti['societa'] = atleti['societa'].apply(lambda x: x.upper())
atleti['categoria'] = atleti['categoria'].apply(lambda x: x.upper())
atleti['id_atleta'] = atleti.apply(lambda row: str(row['cognome']) + '_' + str(row['nome']) + '_' + str(row['sesso']) + '_' + str(row['anno_nascita']), axis=1)
atleti['id_atleta'] = atleti['id_atleta'].apply(lambda x: re.sub(r'\s', '', x))

iscrizioni_individuali = pd.DataFrame()
iscrizioni_individuali['id_societa'] = atleti['societa'].apply(lambda x: re.sub(r'\s', '', x))
iscrizioni_individuali['id_atleta'] = atleti['id_atleta']
iscrizioni_individuali['categoria'] = atleti['categoria']
iscrizioni_individuali['id_iscrizione'] = range(1, len(iscrizioni_individuali) + 1)
iscrizioni_individuali.set_index('id_iscrizione', inplace=True)

def salva_iscrizioni_individuali(df):
    db.execute_query('DELETE FROM iscrizioni_individuali', None)
    for index, row in df.iterrows():
        # Aggiunta degli indici alla lista dei valori
        values_with_indices = [index] + [None if (pd.isna(value)) else value for value in row]
        
        # Aggiunta degli indici alla lista dei nomi delle colonne
        columns_with_indices = ['id_iscrizione'] + df.columns.tolist()
        
        # Creazione della query di inserimento con i parametri
        sql = f"INSERT INTO iscrizioni_individuali ({', '.join(columns_with_indices)}) VALUES ({', '.join(['%s' for _ in range(len(values_with_indices))])})"
        
        # Esecuzione della query utilizzando la funzione execute_query
        db.execute_query(sql, tuple(values_with_indices))

salva_iscrizioni_individuali(iscrizioni_individuali)
atleti = atleti.drop(columns=['societa', 'categoria'])

temp_df1 = df_tot_squadra_A.drop(columns=['N_ISCRITTO_SOCIETA'])
temp_df1.rename(inplace=True, columns={'COGNOME': 'cognome', 'NOME': 'nome', 'SESSO': 'sesso', 'ANNO NASCITA': 'anno_nascita', 'CINTURA': 'cintura', 'SOCIETA': 'societa', 'CATEGORIA': 'categoria'})
temp_df1['sesso'] = temp_df1['sesso'].apply(lambda x: x[0].upper())
temp_df1['cognome'] = temp_df1['cognome'].apply(lambda x: x.upper())
temp_df1['nome'] = temp_df1['nome'].apply(lambda x: x.upper())
temp_df1['cintura'] = temp_df1['cintura'].apply(lambda x: x.upper())
temp_df1['societa'] = temp_df1['societa'].apply(lambda x: x.upper())
temp_df1['categoria'] = temp_df1['categoria'].apply(lambda x: x.upper())
temp_df1['id_atleta'] = temp_df1.apply(lambda row: str(row['cognome']) + '_' + str(row['nome']) + '_' + str(row['sesso']) + '_' + str(row['anno_nascita']), axis=1)
temp_df1['id_atleta'] = temp_df1['id_atleta'].apply(lambda x: re.sub(r'\s', '', x))
temp_df1['squadra'] = '_SQUADRA_A'

temp_df2 = df_tot_squadra_A.drop(columns=['N_ISCRITTO_SOCIETA'])
temp_df2.rename(inplace=True, columns={'COGNOME': 'cognome', 'NOME': 'nome', 'SESSO': 'sesso', 'ANNO NASCITA': 'anno_nascita', 'CINTURA': 'cintura', 'SOCIETA': 'societa', 'CATEGORIA': 'categoria'})
temp_df2['sesso'] = temp_df2['sesso'].apply(lambda x: x[0].upper())
temp_df2['cognome'] = temp_df2['cognome'].apply(lambda x: x.upper())
temp_df2['nome'] = temp_df2['nome'].apply(lambda x: x.upper())
temp_df2['cintura'] = temp_df2['cintura'].apply(lambda x: x.upper())
temp_df2['societa'] = temp_df2['societa'].apply(lambda x: x.upper())
temp_df2['categoria'] = temp_df2['categoria'].apply(lambda x: x.upper())
temp_df2['id_atleta'] = temp_df2.apply(lambda row: str(row['cognome']) + '_' + str(row['nome']) + '_' + str(row['sesso']) + '_' + str(row['anno_nascita']), axis=1)
temp_df2['id_atleta'] = temp_df2['id_atleta'].apply(lambda x: re.sub(r'\s', '', x))
temp_df2['squadra'] = '_SQUADRA_B'

iscrizioni_squadre_temp = pd.concat([temp_df1, temp_df2], axis=0)
iscrizioni_squadre_temp['id_squadra'] = iscrizioni_squadre_temp['societa'].apply(lambda x: re.sub(r'\s', '', x)) + iscrizioni_squadre_temp['squadra']
iscrizioni_squadre = iscrizioni_squadre_temp[['id_squadra', 'id_atleta']]
iscrizioni_squadre['id_iscrizione'] = range(1, len(iscrizioni_squadre) + 1)
iscrizioni_squadre.set_index('id_iscrizione', inplace=True)

temp_df1 = temp_df1.drop(columns=['societa', 'categoria', 'squadra'])
temp_df2 = temp_df2.drop(columns=['societa', 'categoria', 'squadra'])
atleti = pd.concat([atleti, temp_df1, temp_df2], axis=0)

atleti.drop_duplicates(subset='id_atleta', inplace=True)
atleti.set_index('id_atleta', inplace=True)
atleti = atleti.sort_index()
atleti['n_ordine'] = range(1, len(atleti) + 1)

def salva_atleti(df):
    db.execute_query('DELETE FROM atleti', None)
    for index, row in df.iterrows():
        # Aggiunta degli indici alla lista dei valori
        values_with_indices = [index] + [None if (pd.isna(value)) else value for value in row]
        
        # Aggiunta degli indici alla lista dei nomi delle colonne
        columns_with_indices = ['id_atleta'] + df.columns.tolist()
        
        # Creazione della query di inserimento con i parametri
        sql = f"INSERT INTO atleti ({', '.join(columns_with_indices)}) VALUES ({', '.join(['%s' for _ in range(len(values_with_indices))])})"
        
        # Esecuzione della query utilizzando la funzione execute_query
        db.execute_query(sql, tuple(values_with_indices))
        
salva_atleti(atleti)


In [43]:
iscrizioni_individuali

,id_societa,id_atleta,categoria,id_iscrizione
ID_ISCRITTO,,,,
1,DOJOSHOTOKANTREVISO,PESANTE_GIULIA_F_2015,ELEFANTE,1
2,DOJOSHOTOKANTREVISO,NASINI_ANTONIO_M_2014,ELEFANTE,2
3,DOJOSHOTOKANTREVISO,VETTOR_PIETRO_M_2017,GIRAFFA,3
4,DOJOSHOTOKANTREVISO,D'IORIO_FRANCESCO_M_2016,ZEBRE,4
5,DOJOSHOTOKANTREVISO,VASSOTTO_ALICE_F_2016,ELEFANTE,5
...,...,...,...,...
264,TSKS,BALDOVIN_PIETRO_M_2015,GIRAFFA,264
265,TSKS,ESEGIO_CARLOTTA_F_2010,TIGRE,265
266,TSKS,FIORI_EMMA_F_2010,TIGRE,266
